### JAI SHREE RAM

Stage 2: Predicting Length of Stay

In [256]:
import pandas as pd
import numpy as np

In [257]:
#import admisson data
df = pd.read_csv('admission_data.csv')

In [258]:
df

,PID,AGE,GENDER,SMOKING,ALCOHOL,HTN,CAD,PRIOR CMP,HB,TLC,...,VT,PSVT,CONGENITAL,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,TARGET
0,506,50,M,0,0,1,1,0,13.1,10.5,...,0,0,0,0,0,0,0,0,0,4
1,798,71,M,0,0,0,1,0,12.9,8.5,...,0,0,0,0,0,0,0,0,0,2
2,798,71,M,0,0,0,1,0,12.9,8.5,...,0,0,0,0,0,0,0,0,0,2
3,798,72,M,0,0,0,1,0,11.4,6.6,...,0,0,0,0,0,0,0,0,0,6
4,989,71,M,0,0,1,1,0,10.7,14.1,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15752,4888286,68,M,0,0,0,1,0,12.2,6.7,...,0,0,0,0,0,0,0,0,0,5
15753,4888926,65,F,0,0,1,1,0,11.3,11.3,...,0,0,0,0,0,0,0,0,0,2
15754,5711587,60,F,0,0,0,0,1,10.1,9.8,...,0,0,0,0,0,0,0,0,0,11
15755,6408503,68,F,0,0,0,1,0,9.8,17.3,...,0,0,0,0,0,0,1,0,0,11


In [259]:
#find correlation between all the variables
corr_matrix = df.corr()
#remove correlated variables above 0.80
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# Find features with correlation greater than 0.80
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
# Drop features 
df.drop(to_drop, axis=1, inplace=True)

C:\Users\purja\AppData\Local\Temp\ipykernel_17596\997048037.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.corr()


In [260]:
#list unique values in chest infection
df['CHEST INFECTION'].unique()
#drop rows with '\\' in chest infection
df = df[df['CHEST INFECTION'] != '\\']

In [261]:
df.dropna(inplace=True)
df = df[df['PID'] != 'NILL']
df = df[(df != 'EMPTY').all(1)]
df['GENDER'] = np.where(df['GENDER'] == 'M', 1, 0)
#convert PID to int
df['PID'] = df['PID'].astype(int)
#convert HB and TLC to float 
df['HB'] = df['HB'].astype(float)
df['TLC'] = df['TLC'].astype(float)
#convert Platelets, glucose and chest infection to int
# df['PLATELETS'] = df['PLATELETS'].astype(float)
# df['GLUCOSE'] = df['GLUCOSE'].astype(float)
df['CHEST INFECTION'] = df['CHEST INFECTION'].astype(int)


C:\Users\purja\AppData\Local\Temp\ipykernel_17596\3592552018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [262]:
df.dtypes

PID                            int32
AGE                            int64
GENDER                         int32
SMOKING                        int64
ALCOHOL                        int64
HTN                            int64
CAD                            int64
PRIOR CMP                      int64
HB                           float64
TLC                          float64
PLATELETS                     object
GLUCOSE                       object
RAISED CARDIAC ENZYMES         int64
SEVERE ANAEMIA                 int64
ANAEMIA                        int64
STABLE ANGINA                  int64
ACS                            int64
STEMI                          int64
ATYPICAL CHEST PAIN            int64
HEART FAILURE                  int64
VALVULAR                       int64
CHB                            int64
SSS                            int64
AF                             int64
VT                             int64
PSVT                           int64
CONGENITAL                     int64
N

In [265]:
#list unique values and counts of PID in descending order
df['PID'].value_counts()

256628     17
273016     11
325715     11
177926     11
156041      9
           ..
353323      1
353380      1
353634      1
353771      1
5711587     1
Name: PID, Length: 11371, dtype: int64

In [291]:
#check if input PID already exists in DF return TARGET variable
def check_pid(pid):
    if pid in df['PID'].values:
        #return mean of all values for PID
        return df[df['PID'] == pid]['TARGET'].mean().round()
    else:
        return 0

In [272]:
#regression model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#split data into train and test and drop Target and PID variable from X
X = df.drop(['TARGET','PID'], axis=1)
y = df['TARGET']


In [294]:
#fit the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)
model = LinearRegression().fit(X_train, y_train)
#predict the model and convert to int 
y_pred = model.predict(X_test)
#calculate accuracy
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)
#accuracy_score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred.round())


0.13065522620904838

In [295]:
#predict the model
test = np.array([69,1,0,0,0,1,0,9.6,16.9,421,110,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
test = test.reshape(1, -1)
inc_pid = 1
y_pred = model.predict(test).astype(int).round()
((y_pred[0] + check_pid(inc_pid))/2).round()


c:\Users\purja\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


4.0

In [296]:
#list data types
df.dtypes

PID                            int32
AGE                            int64
GENDER                         int32
SMOKING                        int64
ALCOHOL                        int64
HTN                            int64
CAD                            int64
PRIOR CMP                      int64
HB                           float64
TLC                          float64
PLATELETS                     object
GLUCOSE                       object
RAISED CARDIAC ENZYMES         int64
SEVERE ANAEMIA                 int64
ANAEMIA                        int64
STABLE ANGINA                  int64
ACS                            int64
STEMI                          int64
ATYPICAL CHEST PAIN            int64
HEART FAILURE                  int64
VALVULAR                       int64
CHB                            int64
SSS                            int64
AF                             int64
VT                             int64
PSVT                           int64
CONGENITAL                     int64
N

In [197]:
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [255]:
# Training the network
model = models.Sequential()
cols = X_train.shape[1]
input_shape = (cols,)
model.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=input_shape))
# model.add(layers.BatchNormalization())
model.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='relu'))
print(model.summary())

# FIT / TRAIN model on training daa
NumEpochs = 20
BatchSize = 8

X_train = np.asarray(X_train).astype('float32')
# Compile the model and fit it on the training data
model.compile(optimizer=optimizers.Adam(lr=1e-4), loss='mean_absolute_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=NumEpochs, batch_size=BatchSize, validation_data=(X_test, y_test))

# Predicting on the test dataset
results = model.evaluate(X_test, y_test)
print("_"*100)
print("Test Loss and MAE")
print("results ", results)


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 1024)              33792     
                                                                 
 dense_46 (Dense)            (None, 1024)              1049600   
                                                                 
 dropout_18 (Dropout)        (None, 1024)              0         
                                                                 
 dense_47 (Dense)            (None, 1024)              1049600   
                                                                 
 dropout_19 (Dropout)        (None, 1024)              0         
                                                                 
 dense_48 (Dense)            (None, 1024)              1049600   
                                                                 
 dropout_20 (Dropout)        (None, 1024)            

None
Epoch 1/20
 766/1191 [==================>...........] - ETA: 25s - loss: 7.4895 - mae: 6.6255

KeyboardInterrupt: 